In [6]:
import requests
import time
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import re
import time
from selenium.webdriver.common.by import By

keyword = '세탁기'
keys = []
page_no = 1
columns = ['title', 'link', 'image', 'price', 'maker', 'category1', 'category2']
reviews_lists = []

with open('API_KEY.txt', 'r') as key:
    for line in key.readlines():
        text = line.strip()
        if text != '':
            keys.append(text.split(':')[1])

headers = {
    'X-Naver-Client-Id' : keys[0],
    'X-Naver-Client-Secret' : keys[1],
}
url = f'https://openapi.naver.com/v1/search/shop?&query={keyword}'
time.sleep(5.5)
text = requests.get(url, headers=headers)
# driver = webdriver.Chrome()
line = text.json()

reviews = []


for item in line['items']:
    page_no = 1
    
    time.sleep(7.7)
#     item_list.append([item['title'], item['link'], item['image'], item['lprice'], item['maker'], item['category3'], item['category4']])    
    data = requests.get(item['link'])
    soup = BeautifulSoup(data.text)
    
    text = soup.select_one('#wrap > div.product_bridge_product__n_89z > a:nth-child(5)')
    time.sleep(5.5)
    
    driver = webdriver.Chrome()
    driver.get('https://search.shopping.naver.com/catalog/21225654919?NaPm=ct%3Dlgu6spo0%7Cci%3D003949e55b1d3ab8955a8d80f2e6c80d637d8cd4%7Ctr%3Dslsl%7Csn%3D95694%7Chk%3Db4650dcdee991cb7ccbda4393e2a2097dd2d16c9')
    time.sleep(3)
    
    page_no = 1
    
    for _ in range(5):
        if page_no == 11:
            driver.find_element(By.CSS_SELECTOR, '#section_review > div.pagination_pagination__JW7zT > a.pagination_next__3_3ip').click()
            page_no = 2
            
        time.sleep(3.8)
        driver.find_element('xpath', f'//*[@id="section_review"]/div[3]/a[{page_no}]').click()
        
        time.sleep(4.8)
        response = driver.find_elements('xpath', '//*[@id="section_review"]/ul/li/div[2]/div[1]/p')
        
        for review in response:
            reviews.append(review.text)
        
        page_no += 1
    break


In [8]:
df = pd.DataFrame(reviews, columns=['comments'])
df.to_csv('comment.csv', index=False)

,comments
0,예전 엘지세탁기 15 년간 써다가 누전 발생해서 부품이 없어서 수리 못하다가 급하게...
1,일단 설치기사님 너무 친절하셔서 기분이 좋았습니다.\n간혹 불친절한 기사님 이야기를...
2,이전 사용 세탁기는 뒤판에 접지선이 있었는데 새 세탁기는 접지선이 없어요. 접지가 ...
3,일단 소음도 적고 먼지도 잘 걸러져서 좋습니다!\n배송은 주말과 공휴일이 껴있어서 ...
4,"23년썼던 세탁기를 보내고 저렴한 가격에 일반세탁기를 구입했어요,,\n설치도 신속,..."
...,...
95,세탁기 잘 받앗습니다 깔끔하니 좋네요 전에꺼랑 같은용량인데 자리 많이 차지하지안고 ...
96,"감사합니다. 비가 많이 오고 있음에도 불구 하고, 설치해 주셨어요 ^^ 필요한 기능..."
97,가성비 짱입니다. 저렴하게 잘 샀어요 오래 걸린다고 해서 마음놓고 그냥 기다리니 오...
98,흰색이라 깔끔하고좋아요 잘쓸게요


In [ ]:
def tokenize_and_stemming(sentence, tokenizer):
    sentence = re.sub("[^\s0-9a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣]", "", sentence) 
    raw_pos_tagged = tokenizer.pos(sentence, stem=True) # POS Tagging with stemming
    sentence_tokenized = []
    for token, pos in raw_pos_tagged:
        if (len(token) != 1) & (pos in ["Noun", "VerbPrefix", "Verb", "Adverb", "Adjective", "Conjunction", "KoreanParticle"]):
            sentence_tokenized.append(token)  
    return sentence_tokenized
